An example of Petri net that is equivalent to a state machine representing a game character moving between different rooms in response to user input.

This may not be a sensible option if movement mechanics are the same for every location.

In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
class Room(BaseModel):
    name: str
    description: str


class Physical(BaseModel):
    name: str

In [ ]:
class GameExitException(Exception):
    """Raised when the player wants to exit the game."""
    pass


def move_from_room_to_room(character: Physical, destinations: list[str]) -> tuple[Physical, str]:
    """
    Display available destinations and get user choice.
    Returns the character and the name of the chosen destination.
    Type 'exit' to quit the game.
    """
    quit_command = "quit"    
    print(f"\n{character.name} can move to:")
    for i, dest in enumerate(destinations, 1):
        print(f"  {i}. {dest}")
    print(f"  (type '{quit_command}' to quit)")
    
    while True:
        choice = input(f"Enter choice (1-{len(destinations)}): ").strip().lower()
        if choice == quit_command:
            raise GameExitException("Player chose to exit the game.")
        try:
            idx = int(choice) - 1
            if 0 <= idx < len(destinations):
                chosen_dest = destinations[idx]
                print(f"{character.name} moves to {chosen_dest}.")
                return (character, chosen_dest)
        except ValueError:
            pass
        print(f"Please enter a number between 1 and {len(destinations)}, or 'exit' to quit")


def direct_to_chosen_room(movement_info: tuple[Physical, str]) -> dict[str, Physical]:
    """
    Distribution function that routes the character to the chosen destination.
    Returns a dict mapping the destination place name to the character token.
    """
    character, destination = movement_info
    return {destination: character}

In [ ]:
map_graph_nodes_and_edges = [
    # === Entrance Chamber ===
    ListPlaceNode(
        name="Entrance Chamber",
        type=Physical,
        tokens=[Physical(name="character")],
    ),
    ArgumentEdgeToTransition("Entrance Chamber", "Leave Entrance", "character"),
    FunctionTransitionNode(
        name="Leave Entrance",
        function=lambda character: move_from_room_to_room(
            character, destinations=["Hallway"]
        ),
        output_distribution_function=direct_to_chosen_room,
    ),
    ReturnedEdgeFromTransition("Leave Entrance", "Hallway"),

    # === Hallway (hub room with multiple exits) ===
    ListPlaceNode(
        name="Hallway",
        type=Physical,
    ),
    ArgumentEdgeToTransition("Hallway", "Leave Hallway", "character"),
    FunctionTransitionNode(
        name="Leave Hallway",
        function=lambda character: move_from_room_to_room(
            character, destinations=["Entrance Chamber", "Treasure Room"]
        ),
        output_distribution_function=direct_to_chosen_room,
    ),
    ReturnedEdgeFromTransition("Leave Hallway", "Entrance Chamber"),
    ReturnedEdgeFromTransition("Leave Hallway", "Treasure Room"),

    # === Treasure Room ===
    ListPlaceNode(
        name="Treasure Room",
        type=Physical,
    ),
    ArgumentEdgeToTransition("Treasure Room", "Leave Treasure Room", "character"),
    FunctionTransitionNode(
        name="Leave Treasure Room",
        function=lambda character: move_from_room_to_room(
            character, destinations=["Hallway"]
        ),
        output_distribution_function=direct_to_chosen_room,
    ),
    ReturnedEdgeFromTransition("Leave Treasure Room", "Hallway"),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(map_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)


In [ ]:
display(SimpleGraphvizVisualization.graph(executable_pydigraph))

In [ ]:
try:
    async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
        executable_graph=executable_graph,
        executable_pydigraph=executable_pydigraph,
    ):
        clear_output(wait=True)
        print(f"Step {step}")
        display(diagram)
        print(f"Transitions fired: {transitions_fired}")
        if not transitions_fired:
            print("No more transitions to fire. Execution complete.")
            break
        time.sleep(1.0)
        plt.close()
except GameExitException:
    clear_output(wait=True)
    print("Thanks for playing! Game exited.")